In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


In [2]:
# typically takes 30 secs
from metrics import *

time: 22.8 s


In [3]:
database = load_pickle('../big_data/database3.pickle')
len(database)

1992

time: 55.9 ms


In [4]:
class ev(evaluation):
    def __init__(self, filename, tag):
        self.dic = self.load_dic({}, filename, tag)
        self.ori = tag
        self.gens = []

    def ingr_f1(self, root = True):
        value, number = [], []
        for i, v in tqdm.tqdm(self.dic.items()):
            true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
            if root:
                true, pred = self.ingr(true), self.ingr(pred)
            scores = metrics(true, pred)
            value.append(scores.f1())
            number.append(len(set(pred)))
        avg = sum(value)/len(value)
        print(avg)
        print('average ingr number', sum(number)/len(number))
        return avg
    
    def ingr_precision_recall(self, generate = 'ingr'):
        assert generate in ['ingr','instr','human']
        precision, recall = [], []
        for i, v in tqdm.tqdm(self.dic.items()):
            if generate == 'ingr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
                true, pred = self.ingr(true), self.ingr(pred)
            elif generate == 'instr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.gen)]
                true, pred = self.ingr(true), self.instr(pred)
            elif generate == 'human':   
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.ori)]
                true, pred = self.ingr(true), self.instr(pred)
            scores = metrics(true, pred)
            precision.append(scores.precision())
            recall.append(scores.recall())
    
        print('precision', sum(precision)/len(precision))
        print('recall', sum(recall)/len(recall))
        
        
    def jaccard(self, generate = 'ingr'):
        assert generate in ['ingr','instr','human']
        jaccard = []
        for i, v in tqdm.tqdm(self.dic.items()):
            if generate == 'ingr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
                true, pred = self.ingr(true), self.ingr(pred)
            elif generate == 'instr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.gen)]
                true, pred = self.ingr(true), self.instr(pred)
            elif generate == 'human':   
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.ori)]
                true, pred = self.ingr(true), self.instr(pred)
            true, pred = set(true), set(pred)
            
            intersect = len(true & pred)
            similarity = intersect /(len(true)+len(pred) - intersect)
            jaccard.append(similarity)
    
        print('jaccard', sum(jaccard)/len(jaccard))
    
    def instr(self, directions):
        instr = sp.spacy(directions)
        root_instr = []
        for chunk in instr.noun_chunks:
            idx_rootnoun = chunk.end - 1
            str_rootnoun = instr[idx_rootnoun].lemma_
            if str_rootnoun in database:
                root_instr.append(str_rootnoun)
        return root_instr
    
    def ingr(self, lst):
        '''
        Args: lst: a list of ingredient names
        used when len(lst) must equal to root_match
        '''
        hl = [[{'text':x, 'highlight': None} for x in i.split(' ')] for i in lst]
        root_match = []
        for i, ingr in enumerate(lst):
            if ' ' not in ingr:
                hl[i][0]['highlight'] = 'wrong'
                doc = sp.spacy(ingr)
                root_match.append(doc[0].lemma_)
            else:
                phrase = 'Mix the %s and water.'%ingr
                doc = sp.spacy(phrase)
                
                last_chunk = None
                for chunk in doc.noun_chunks:
                    if chunk.text != 'water':
                        last_chunk = chunk
                if not last_chunk:
                    root_match.append('CANNOT_DETECT')
                else:
                    found = False
                    for j, word in enumerate(hl[i]):
                        if doc[last_chunk.end - 1].text in word['text']:
                            hl[i][j]['highlight'] = 'wrong' 
                            root_match.append(doc[last_chunk.end - 1].lemma_)
                            found = True
                            break
                    if not found:
                        root_match.append('CANNOT_DETECT')
                        
        assert len(root_match) == len(lst)
        return root_match

    '''
    exporting data
    '''
    def to_bleu(self):
        to_write = {'%s_i'%(self.ori):'',
                    '%s_i'%(self.gen):'',
                    '%s_d'%(self.ori):'',
                    '%s_d'%(self.gen):''}
        
        for i, v in self.dic.items():
            to_write['%s_i'%(self.ori)] += self.add_space(' $ '.join(v['%s_ingr'%(self.ori)]))+ ' $ \n'
            to_write['%s_i'%(self.gen)] += self.add_space(' $ '.join(v['%s_ingr'%(self.gen)])) + ' $ \n'
            
            to_write['%s_d'%(self.ori)] += self.add_space(v['%s_instr'%(self.ori)])+ '\n'
            to_write['%s_d'%(self.gen)] += self.add_space(v['%s_instr'%(self.gen)])+ '\n'
        
        for k, v in to_write.items():
            save('../../to_gpt2/generation_%s.txt'%(k), v ,overwrite = True)
        !eval {"perl multi-bleu.perl ../../to_gpt2/generation_%s_i.txt < ../../to_gpt2/generation_%s_i.txt" %(self.ori, self.gen)}
        !eval {"perl multi-bleu.perl ../../to_gpt2/generation_%s_d.txt < ../../to_gpt2/generation_%s_d.txt" %(self.ori, self.gen)}
    
        !eval {"rouge -f ../../to_gpt2/generation_%s_i.txt ../../to_gpt2/generation_%s_i.txt --avg"%(self.ori, self.gen)}
        !eval {"rouge -f ../../to_gpt2/generation_%s_d.txt ../../to_gpt2/generation_%s_d.txt --avg"%(self.ori, self.gen)}
        
        print()

time: 48.7 ms


## Table 1a

In [ ]:
data = ev('../../to_gpt2/recipe1M_1218/val/y/', 'ori')
data.append_dic('../../to_gpt2/val/generation_1220_k1_val/', 'k1')
data.append_dic('../../to_gpt2/val/generation_1220_k3_val/', 'k3')
data.append_dic('../../to_gpt2/val/generation_1220_k5_val/', 'k5')
data.append_dic('../../to_gpt2/val/generation_1220_k10_val/', 'k10')
data.append_dic('../../to_gpt2/val/generation_1220_k30_val/', 'k30')
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.ingr_f1(root=True)

load ../../to_gpt2/recipe1M_1218/val/y/
load ../../to_gpt2/val/generation_1220_k1_val/
load ../../to_gpt2/val/generation_1220_k3_val/
load ../../to_gpt2/val/generation_1220_k5_val/
load ../../to_gpt2/val/generation_1220_k10_val/
load ../../to_gpt2/val/generation_1220_k30_val/


  0%|          | 1/4000 [00:00<11:54,  5.60it/s]

k1


  0%|          | 1/4000 [00:00<10:04,  6.62it/s]

0.7852302818896786
average ingr number 7.62425
k3


 40%|████      | 1614/4000 [04:20<07:05,  5.61it/s]

In [ ]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.to_bleu()

In [ ]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.instr_tree(stem_only = False)

## Table 1b

In [ ]:
data = ev('../../to_gpt2/recipe1M_1218/test/y/', 'ori')
data.append_dic('../../to_gpt2/generation_1220_k3_test/', '117M')
data.append_dic('../../to_gpt2/generation_scratch_k3_test/', 'scratch')
data.append_dic('../../to_gpt2/generation_medium_k3_test/', '345M')
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.ingr_f1(root=True)

In [ ]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.to_bleu()

In [ ]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.instr_tree(stem_only = False)

## Jaccard similarity

In [ ]:
data = ev('../../to_gpt2/recipe1M_1218/test/y/', 'ori')
data.append_dic('../../to_gpt2/generation_1220_k3_test/', '117M')
data.jaccard(generate = 'ingr')
data.jaccard(generate = 'instr')
data.jaccard(generate = 'human')